### Imports & dataset

In [1]:
# --- Imports ---
import torch
import numpy as np
import inr_sos
from inr_sos import DATA_DIR, clear_cache
from inr_sos.utils.data import USDataset
from inr_sos.utils.config import ExperimentConfig
from inr_sos.evaluation.pipeline import run_grid_comparison
from inr_sos.evaluation.sweep_agent import (
    get_sweep_config,
    run_sweep_agent
)
import wandb
# --- Load Global Data ---
inverse_data_file = DATA_DIR + "/DL-based-SoS/train-VS-8pairs-IC-081225.mat"
param_grid_file =  DATA_DIR  + "/DL-based-SoS/forward_model_lr/grid_parameters.mat"
analytical_data = inr_sos.load_mat(DATA_DIR + "/DL-based-SoS/train_IC_10k_l2rec_l1rec_imcon.mat")
dataset = USDataset(inverse_data_file, param_grid_file)

INFO:root:Loading L-Matrix from data file: /mnt/asgard0/data/haben/data/DL-based-SoS/train-VS-8pairs-IC-081225.mat
INFO:root:Loading L-Matrix from data file...
INFO:root:L-Matrix loaded successfully with shape: torch.Size([131072, 4096])
INFO:root:Dataset initialized with 10000 samples.


In [3]:
def run_experiment(dataset: USDataset, indices : list[int], config: ExperimentConfig):
    import copy   
    from inr_sos.models.mlp import FourierMLP, ReluMLP
    from inr_sos.models.siren import SirenMLP
    from inr_sos.training.engines import optimize_stochastic_ray_batching, optimize_full_forward_operator, optimize_sequential_views
    from inr_sos.evaluation.pipeline import run_evaluation
    # --- 1. Define the Grid ---
    optimization_engines = {
        "Full_Matrix": optimize_full_forward_operator,
        "Sequential_SGD": optimize_sequential_views,
        "Ray_Batching": optimize_stochastic_ray_batching
    }
    
    model_architectures = {
        "ReluMLP": ReluMLP,
        "FourierMLP": FourierMLP,
        "SirenMLP": SirenMLP
    }
    
    # --- 2. Run the Gauntlet ---
    for method_name, engine_func in optimization_engines.items():
        print(f"\n{'='*60}")
        print(f" LAUNCHING OPTIMIZER: {method_name}")
        print(f"{'='*60}")
        
        for model_name, model_cls in model_architectures.items():
            print(f"\n---> Testing Backbone: {model_name}")
            
            # Clone config so they don't overwrite each other
            cfg_clone = copy.deepcopy(config)
            
            # This is where your W&B architecture magic happens!
            cfg_clone.experiment_group = method_name  
            cfg_clone.model_type = model_name         
            
            # Run the pipeline
            run_evaluation(
                dataset=dataset,
                model_class=model_cls, 
                train_engine=engine_func,
                config=cfg_clone,
                target_indices=indices,
                use_wandb=True # This ensures the loss curves stream to W&B
            )

###  Base config

In [2]:
base_config = ExperimentConfig(
    project_name="INR-SoS-Recon",
    in_features=2,
    hidden_features=256,
    hidden_layers=3,
    mapping_size=64,
    scale=0.6,
    omega=30.0,
    lr=1e-4,
    steps=2000,
    epochs=150,
    batch_size=4096,
    tv_weight=0.0,
    reg_weight=0.0,
)

# Pick a handful of samples for the grid comparison
indices = np.random.choice(len(dataset), size=3, replace=False).tolist()
print("Evaluating on samples:", indices)

Evaluating on samples: [8052, 7863, 2923]


###  Full grid comparison
(all 3 methods × all 3 models) each (method/model) combo gets its own W&B group. One master SUMMARY run holds the comparison table.

In [3]:
results = run_grid_comparison(
    dataset=dataset,
    target_indices=indices,
    base_config=base_config,
    use_wandb=True,
)

# Quick console summary
print("\n── Grid Results ──────────────────────────────────")
print(f"{'Method':<20} {'Model':<12} {'MAE_mean':>10} {'SSIM_mean':>10}")
print("-" * 55)
for (method, model), vals in sorted(results.items()):
    mae  = np.mean(vals["MAE"])
    ssim = np.mean(vals["SSIM"])
    print(f"{method:<20} {model:<12} {mae:>10.4f} {ssim:>10.4f}")

INFO:root:Grid: 3 methods × 3 models × 3 samples = 27 runs
INFO:root:
  Full_Matrix/ReluMLP


wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /home/haben/.netrc.
wandb: Currently logged in as: habenhadush (habenhadush-uppsala-universitet) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


INFO:root:
--- optimize_full_forward_operator: Training ReluMLP (full-matrix) on cuda ---


Loss (us^2): 0.0000: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:19<00:00, 104.71it/s, method=optimize_full_forward_operator, model=ReluMLP]


Data Loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Final/CNR,▁
Final/MAE,▁
Final/RMSE,▁
Final/SSIM,▁
Learning Rate,█████▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁
Total Loss,██▆▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Data Loss,0.0
Final/CNR,3.22137
Final/MAE,3.24245
Final/RMSE,4.79832


...

INFO:root:
--- optimize_full_forward_operator: Training FourierMLP (full-matrix) on cuda ---


Loss (us^2): 0.0000: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:18<00:00, 110.95it/s, method=optimize_full_forward_operator, model=FourierMLP]


Data Loss,█▅▅▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Final/CNR,▁
Final/MAE,▁
Final/RMSE,▁
Final/SSIM,▁
Learning Rate,█████████▇▇▇▇▇▇▇▇▆▆▆▅▄▄▄▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁
Total Loss,█▅▅▅▅▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Data Loss,3e-05
Final/CNR,3.7261
Final/MAE,6.25725
Final/RMSE,11.16551


INFO:root:  [Full_Matrix_FourierMLP_Sample_2923] MAE=6.257 RMSE=11.166 SSIM=0.7219 CNR=3.726
INFO:root:
  Full_Matrix/SirenMLP


TypeError: SirenMLP.__init__() got an unexpected keyword argument 'omega'. Did you mean 'omega_0'?

### Bayesian sweep for the best-looking (method, model) pair
After inspecting the grid results above, set method and model_type to the combo you want to tune, then run this cell.

In [ ]:
TUNE_METHOD     = "Full_Matrix"   # <-- change after inspecting Cell 3
TUNE_MODEL_TYPE = "FourierMLP"    # <-- change after inspecting Cell 3
N_SWEEP_RUNS    = 30              # total Bayesian trials

sweep_cfg = get_sweep_config(
    model_type=TUNE_MODEL_TYPE,
    method=TUNE_METHOD,
    metric_goal="MAE_mean",
    metric_direction="minimize",
)

sweep_id = wandb.sweep(sweep_cfg, project=base_config.project_name)
print(f"Sweep created: {sweep_id}")
print(f"  View at: https://wandb.ai/{wandb.api.default_entity}/{base_config.project_name}/sweeps/{sweep_id}")

# Launch the agent in-process (blocks until n_runs are done)
# For parallel runs across GPUs, copy the wandb agent CLI command instead:
#   wandb agent <entity>/<project>/<sweep_id>
run_sweep_agent(
    sweep_id=sweep_id,
    dataset=dataset,
    target_indices=indices,
    base_config=base_config,
    model_type=TUNE_MODEL_TYPE,
    method=TUNE_METHOD,
    n_runs=N_SWEEP_RUNS,
)

### Evaluate best sweep config on a larger held-out set
After the sweep completes, grab best_config from the W&B API and re-run the grid on more samples.

In [ ]:
api = wandb.Api()
sweep = api.sweep(f"{wandb.api.default_entity}/{base_config.project_name}/{sweep_id}")

best_run = min(sweep.runs, key=lambda r: r.summary.get("MAE_mean", float("inf")))
print(f"\nBest run: {best_run.name}  MAE_mean={best_run.summary['MAE_mean']:.4f}")
print("Best hyperparams:", dict(best_run.config))

# Patch base_config with best hyperparams
best_cfg = ExperimentConfig(
    **{**base_config.to_dict(), **best_run.config}
)

# Evaluate on a larger held-out set
holdout_indices = np.random.choice(len(dataset), size=10, replace=False).tolist()

from inr_sos.evaluation.comparison import run_grid_comparison
# Run just the winning (method, model) combo on the holdout set
from inr_sos.training.engines import optimize_full_forward_operator   # or whichever won
from inr_sos.models.mlp import FourierMLP

holdout_results = run_grid_comparison(
    dataset=dataset,
    target_indices=holdout_indices,
    base_config=best_cfg,
    engines={TUNE_METHOD: optimize_full_forward_operator},
    models={TUNE_MODEL_TYPE: FourierMLP},
    use_wandb=True,
)